In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")


# class MyQueryEndEventHandler(BaseEventHandler):
#     @classmethod
#     def class_name(cls) -> str:
#         """Class name."""
#         return "MyQueryEndEventHandler"

#     def handle(self, event) -> None:
#         """Logic for handling event."""
#         if isinstance(event, QueryEndEvent):
#             print(self.class_name())
#             with open("log.txt", "a") as f:
#                 f.write(str(event))
#                 f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x1422ae8f0>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='BaseRetriever.retrieve-91639d99-5285-49e9-89ca-13ee3b176aee', parent_id='RetrieverQueryEngine._query-9fb670e4-787b-42cf-8f03-84109f647971', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 35059), end_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 195471), duration=0.160412),
 SimpleSpan(id_='LLM.predict-bdb613f4-466e-4b25-8126-8aa91526b25c', parent_id='Refine.get_response-807d5060-c340-4a9e-b660-fb0c6a4dafd9', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 201071), end_time=datetime.datetime(2024, 3, 6, 16, 43, 58, 961112), duration=1.760041),
 SimpleSpan(id_='Refine.get_response-807d5060-c340-4a9e-b660-fb0c6a4dafd9', parent_id='CompactAndRefine.get_response-eaa177a5-61c4-4e75-bcc9-bf82b300400e', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 198646), end_time=datetime.datetime(2024, 3, 6, 16, 43, 58, 961409), duration=1.762763),
 SimpleSpan(id_='CompactAndRefine.get_response-eaa177a5-61c4-4e75-bcc9-bf82b300400e', parent_id='BaseSynthesizer.synthesize-

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='BaseRetriever.retrieve-91639d99-5285-49e9-89ca-13ee3b176aee', parent_id='RetrieverQueryEngine._query-9fb670e4-787b-42cf-8f03-84109f647971', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 35059), end_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 195471), duration=0.160412),
 SimpleSpan(id_='LLM.predict-bdb613f4-466e-4b25-8126-8aa91526b25c', parent_id='Refine.get_response-807d5060-c340-4a9e-b660-fb0c6a4dafd9', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 201071), end_time=datetime.datetime(2024, 3, 6, 16, 43, 58, 961112), duration=1.760041),
 SimpleSpan(id_='Refine.get_response-807d5060-c340-4a9e-b660-fb0c6a4dafd9', parent_id='CompactAndRefine.get_response-eaa177a5-61c4-4e75-bcc9-bf82b300400e', start_time=datetime.datetime(2024, 3, 6, 16, 43, 57, 198646), end_time=datetime.datetime(2024, 3, 6, 16, 43, 58, 961409), duration=1.762763),
 SimpleSpan(id_='CompactAndRefine.get_response-eaa177a5-61c4-4e75-bcc9-bf82b300400e', parent_id='BaseSynthesizer.synthesize-

### Very Primitive Tree Viz

In [ ]:
%pip install treelib -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from treelib import Node, Tree

tree = Tree()

In [ ]:
for span in reversed(dispatcher.span_handler.completed_spans[:7]):
    tree.create_node(
        tag=f"{span.id_} ({span.duration})",
        identifier=span.id_,
        parent=span.parent_id,
        data=span.duration,
    )

In [ ]:
print(tree.show(stdout=False))

BaseQueryEngine.query-c46703f1-5312-4852-a58b-559559ca9bea (1.927449)
└── RetrieverQueryEngine._query-9fb670e4-787b-42cf-8f03-84109f647971 (1.926662)
    ├── BaseRetriever.retrieve-91639d99-5285-49e9-89ca-13ee3b176aee (0.160412)
    └── BaseSynthesizer.synthesize-a8fabe88-45d2-4c2e-a817-b68b52211f9a (1.766149)
        └── CompactAndRefine.get_response-eaa177a5-61c4-4e75-bcc9-bf82b300400e (1.765516)
            └── Refine.get_response-807d5060-c340-4a9e-b660-fb0c6a4dafd9 (1.762763)
                └── LLM.predict-bdb613f4-466e-4b25-8126-8aa91526b25c (1.760041)

